In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as _mp
from torchvision import transforms, datasets

import resnet_cifar10
from networks import NetDelta, NetTMR, NetDMR, NetTMROutput
from utils import *

import copy
import time
import pickle
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np

import sysconfig
print(sysconfig.get_paths()['include'])

device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

print('Using device :', device)
#Additional Info when using cuda
if device.type == 'cuda':
    print('GPU          :', torch.cuda.get_device_name(0))
else:
    import cpuinfo
    print("CPU          :", cpuinfo.get_cpu_info()['brand_raw'])

print("Pytorch version: ",torch.__version__)


In [ ]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

test_set = datasets.CIFAR10("cifar10", train=False, download=True, transform=test_transform)

testloaders = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=True)

In [ ]:
run_time = 20

## ResNets

In [ ]:
model_types = [20,32,44,56]
idss =        [[0, 302], [0,346],[0,365],[0,368]]
base_times = []

# base
for id, mod in zip(idss, model_types):
    models = []
    ids = id
    model_name ="resnet"+str(mod)
    models_path = "resnet_models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    print(evaluate_acc(models[0], testloaders))
    evaluate_acc(models[0], testloaders, single=True, time_profiler=True)
    print("finish warming up..")
    model_time = []
    for i in range(run_time):
        model_time.append(evaluate_acc(models[0], testloaders, single=True, time_profiler=True)[-1])
    base_times.append(model_time)
    # break
print(base_times)

In [ ]:
meantime = 0.
for _ in range(10):
    sample_input= next(iter(testloaders))[0][0].unsqueeze(0).to(device)
    if device == "cuda": 
        torch.cuda.synchronize()
    # with torch.no_grad():
    startdmr = time.perf_counter()
    models[0](sample_input)
    if device == "cuda": 
        torch.cuda.synchronize()
    enddmr = time.perf_counter()
    meantime += (enddmr-startdmr)*1000
meantime /= 10
print("mean time: ", meantime)

## TMR

In [ ]:
model_types = [20,32,44,56]
idss =        [[0, 302], [0,346],[0,365],[0,368]]
TMR_times = []
TMR_times2 = []


for id, mod in zip(idss, model_types):
    models = []
    ids = id
    model_name ="resnet"+str(mod)
    models_path = "resnet_models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    print(evaluate_acc(models[0], testloaders))

    modelTMR = NetTMR(models[0])
    modelTMR.to(device)
    modelTMR.eval()
    print("model TMR ready")
    print("accuracy", evaluate_acc(modelTMR, testloaders))

    evaluate_acc(modelTMR, testloaders, single=True, time_profiler=True)
    print("finish warming up..")
    model_time = []
    modelTMR.protected=True
    # modelTMR.detection_time = True #########
    for i in range(run_time):
        # modelTMR.resetTime()
        model_time.append(evaluate_acc(modelTMR, testloaders, single=True, time_profiler=True)[-1])
    TMR_times.append(model_time)
    # TMR_times2.append(np.sum(modelTMR.list_detection_time)) #########
#     break #########
# print(TMR_times)
# print(np.mean(TMR_times2)) #########

# TMR Output

In [ ]:
model_types = [20,32,44,56]
idss =        [[0, 302], [0,346],[0,365],[0,368]]
TMRo_times = []
TMRo_times2 = []

for id, mod in zip(idss, model_types):
    models = []
    ids = id
    model_name ="resnet"+str(mod)
    models_path = "resnet_models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    print(evaluate_acc(models[0], testloaders))

    modelTMRo = NetTMROutput(models[0])
    modelTMRo.to(device)
    modelTMRo.eval()
    print("model TMR ready")
    print("accuracy", evaluate_acc(modelTMRo, testloaders))

    evaluate_acc(modelTMRo, testloaders, single=True, time_profiler=True)
    print("finish warming up..")
    model_time = []
    modelTMRo.protected=True
    # modelTMRo.detection_time = True #########
    for i in range(run_time):
        # modelTMRo.resetTime()
        model_time.append(evaluate_acc(modelTMRo, testloaders, single=True, time_profiler=True)[-1])
    TMRo_times.append(model_time)
    # TMRo_times2.append(np.sum(modelTMRo.list_detection_time)) #########
#     break #########
print(TMRo_times)
# print(np.mean(TMRo_times2)) #########

# DMR

In [ ]:
model_types = [20,32,44,56]
idss =        [[0, 302], [0,346],[0,365],[0,368]]
DMR_times = []
DMR_times2 = []

for id, mod in zip(idss, model_types):
    models = []
    ids = id
    model_name ="resnet"+str(mod)
    models_path = "resnet_models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    print(evaluate_acc(models[0], testloaders))

    modelDMR = NetDMR(models[0], "", sum_model(models[0]))
    modelDMR.to(device)
    modelDMR.eval()
    print("model DMR ready")
    print("accuracy", evaluate_acc(modelDMR, testloaders))

    evaluate_acc(modelDMR, testloaders, single=True, time_profiler=True)
    print("finish warming up..")
    model_time = []
    modelDMR.protected=True
    # modelDMR.detection_time = True
    # modelDMR.recovery_time = True
    modelDMR.debug = False
    for i in range(run_time):
        # modelDMR.resetTime()
        model_time.append(evaluate_acc(modelDMR, testloaders, single=True, time_profiler=True)[-1])
        # DMR_times2.append(np.sum(modelDMR.list_detection_time))
    DMR_times.append(model_time)
    # break
print(DMR_times)
print("Average of Total Inference Time (no fault)    :", "{:.4f}".format(np.mean(DMR_times[0])), "ms")
# print("Average of Total Detection Time (no fault)    :", "{:.4f}".format(np.mean(DMR_times2)), "ms")


# NAPER

In [ ]:
model_types = [20,32,44,56]
idss =        [[0, 302], [0,346],[0,365],[0,368]]
NaPER_times = []
# NaPER_single = []
NaPER_times2 = []

# NAPER ENSEMBLE
for id, mod in zip(idss, model_types):
    models = []
    ids = id
    model_name ="resnet"+str(mod)
    models_path = "resnet_models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    print("model 0 accuracy", evaluate_acc(models[0], testloaders))

    deltas, summs = get_sum_delta(models)
    modelDelta = NetDelta(models, deltas, summs)
    modelDelta.protected=False
    # modelDelta.detection_time = True
    print("model NaPER ready")
    print("ensemble accuracy",ens_evaluate_acc(wrappermodel=modelDelta, testloaders=testloaders))
    print("ensemble 0 accuracy",ens_evaluate_acc(wrappermodel=modelDelta, testloaders=testloaders, limit_model=1))

    print("finish warming up..")
    model_time = []
    modelDelta.protected=True
    for i in range(run_time):
        # modelDelta.resetTime()
        model_time.append(ens_evaluate_acc(wrappermodel=modelDelta, testloaders=testloaders, single=True, time_profiler=True)[-1])
    NaPER_times.append(model_time)
    # NaPER_times2.append(np.sum(modelDelta.list_detection_time))

    # model_time = []
    # for i in range(run_time):
    #     model_time.append(ens_evaluate_acc(wrappermodel=modelDelta, testloaders=testloaders, single=True,
    #                                        time_profiler=True, limit_model=1)[-1])
    # NaPER_single.append(model_time)
    # break
print(NaPER_times)
# print(NaPER_single)
# print(np.mean(NaPER_times2))

# Ensemble ResNet

In [ ]:
model_types = [20,32,44,56]
idss =        [[0, 302], [0,346],[0,365],[0,368]]
EnsRes_time = []

for id, mod in zip(idss, model_types):
    models = []
    ids = id
    model_name ="resnet"+str(mod)
    models_path = "resnet_models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    print(evaluate_acc(models[0], testloaders))

    # deltas, summs = get_sum_delta(models)
    # modelDelta = NetDelta(models, ids, mod, deltas, summs)
    print("model ensemble ready")
    print("accuracy",ens_evaluate_acc(models=models, testloaders=testloaders))
    ens_evaluate_acc(models=models, testloaders=testloaders, single=True, time_profiler=True)
    ens_evaluate_acc(models=models, testloaders=testloaders, single=True, time_profiler=True)
    print("finish warming up..")
    model_time = []
    for i in range(run_time):
        model_time.append(ens_evaluate_acc(models=models, testloaders=testloaders, single=True, time_profiler=True)[-1])
    EnsRes_time.append(model_time)
print(EnsRes_time)